# Procesos Gaussianos y performance de código

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.optimize import fmin_l_bfgs_b as fmin  # minimizar
import time  # medir tiempo
from numba import jit  # decorador para compilar funciones a C

__Configuración para los gráficos__

In [ ]:
sns.set_context('notebook', font_scale=2)
sns.set_style('white')
plt.rcParams['figure.figsize'] = (14, 5)

# Mediciones de tiempo

In [ ]:
def fill_matrix(n):
    """ Llena una matriz aleatoria de n x n """
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            A[i, j] = np.random.random()
    return A

In [ ]:
ti = time.time()
fill_matrix(5000)
dt = time.time() - ti
print('Tiempo ejecución {} s'.format(dt))

In [ ]:
%timeit fill_matrix(5000)

__Intentemos mejorarla__

In [ ]:
@jit
def fill_matrix2(n):
    """ Llena una matriz aleatoria de n x n """
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            A[i, j] = np.random.random()
    return A

In [ ]:
ti = time.time()
fill_matrix2(5000)
dt = time.time() - ti
print('Tiempo ejecución {} s'.format(dt))

In [ ]:
%timeit fill_matrix2(5000)

__podemos mejorar la función aún mas?__

In [ ]:
def fill_matrix3(n):
    """ Llena una matriz aleatoria de n x n """
    A = np.random.random(size=(n, n))
    return A

In [ ]:
ti = time.time()
fill_matrix3(5000)
dt = time.time() - ti
print('Tiempo ejecución {} s'.format(dt))

In [ ]:
%timeit fill_matrix3(5000)

__hay que tener cuidado con @jit !__

In [ ]:
cte = 10

In [ ]:
def fn_nojit(x):
    return x + cte

In [ ]:
@jit
def fn_conjit(x):
    return x + cte

In [ ]:
print('Sin jit ', fn_nojit(3))
print('Con jit ', fn_conjit(3))

__modifiquemos `cte`__

In [ ]:
cte = 50

In [ ]:
print('Sin jit ', fn_nojit(3))
print('Con jit ', fn_conjit(3))

__¿Por que ocurre esto?__

# Procesos Gaussianos

In [ ]:
def K_SE(a, b, gamma=1. / 2, sigma=1):
    """
    Squared Exponential kernel
    Returns the gram matrix given by the kernel
    k(a,b) = sigma**2*exp(-gamma*(a-b)**2)
    Note that: gamma = 1 /(2*lengthscale**2)
    
    Inputs:
    a:(numpy array)   Array length n_a with first input
    b:(numpy array)   Array length n_b with second input
    gamma:(float)     Kernel parameter
    sigma:(float)     Kernel parameter, signal variance

    Returns:
    (numpy array) n_a X n_b gram matrix where element
    [i,j] = k(a[i], b[j])
    """
    # transform to array if a single point
    if np.ndim(a) == 0: a = np.array([a])
    if np.ndim(b) == 0: b = np.array([b])
    # create matrix
    gram = np.zeros((len(a), len(b)))
    # compute
    for i, va in enumerate(a):
        for j, vb in enumerate(b):
            gram[i, j] = np.exp(-gamma * (va - vb)**2)
    # condition if a single point
    if (len(a) == 1) or (len(b) == 1):
        return gram.reshape(-1)
    else:
        return gram


def like_SE(theta, y, t):
    """
    Computes negative log-likelihood when using SE kernel
    Designed for training w.r.t log of hyperparameters, not imposing restrictions on solver
    
    Inputs:
    theta:(numpy array) Kernel parameters, sigma_noise | gamma | sigma_signal
    y:(numpy array)     Array of observations-length n
    t:(numpy array)     Array of time-length n
    
    Returns:
    (float) Computed nll
    """
    sigma_noise, gamma_1, sig_1 = np.exp(theta)
    Gram = K_SE(
        t, t, gamma=gamma_1,
        sigma=sig_1) + sigma_noise**2 * np.identity(len(t))
    # inverse with cholesky
    cGg = np.linalg.cholesky(Gram)
    invGram = np.linalg.inv(cGg.T) @ np.linalg.inv(cGg)
    # nll
    nll = 2 * np.log(np.diag(cGg)).sum() + y.T @ (invGram @ y) 
    return 0.5 * nll + 0.5 * len(y) * np.log(2 * np.pi)

## Cargar datos

In [ ]:
np.random.seed(123)
data = np.loadtxt('wool.csv', delimiter=',')

t = data[:, 0]
y = data[:, 1]

prop = 0.15
i_obs = np.random.choice(np.arange(len(t)), int(prop * len(t)), replace=False)

t_obs = t[i_obs]
y_obs = y[i_obs]
n_obs = len(i_obs)

In [ ]:
plt.plot(t, y, label='Señal real')
plt.scatter(t_obs, y_obs, c='r', label='Obs')
plt.legend()

## Prueba a mano

In [ ]:
# hand-chosen hyperparameter
sigma_n_GP_test = .001
gamma_test = .01
sigma_test = .1

print('Negative log-likelihood para hiperámetros escogidos: ',
      like_SE([sigma_n_GP_test, gamma_test, sigma_test], y_obs, t_obs))

In [ ]:
# process prior covariance 
cov = K_SE(t, t, gamma=gamma_test, sigma=sigma_test)

# observations prior covariance 
cov_obs = K_SE(t_obs, t_obs, gamma=gamma_test, sigma=sigma_test)

# K(x*, x) kernel evaluation between obs and test points
K_star = K_SE(t, t_obs, gamma=gamma_test, sigma=sigma_test)

# cholesky
cGg = np.linalg.cholesky(cov_obs + sigma_n_GP_test**2 * np.identity(n_obs))

# inverse covariance matrix (gram)
invGramg = np.linalg.inv(cGg.T) @ np.linalg.inv(cGg)

# temporal matrix K(x*, x) (K(x, x) + sigma * I)^-1
temp = K_star @ invGramg
# posterior mean
m_post = temp @ y_obs
# posterior covariance
sigma_post = np.diag(cov - temp @ K_star.T)
std_dev = np.sqrt(sigma_post)

# plot estimation, obs and real data
plt.plot(t, m_post, c='#00BFFF', lw=4, label='Media de la posterior')
plt.fill_between(
    t,
    m_post - 2 * std_dev,
    m_post + 2 * std_dev,
    facecolor='blue',
    alpha=0.2,
    label='95% IC')
plt.plot(t, y, color='black', ls='--', lw=2, label='Señal real')
plt.plot(t_obs, y_obs, 'r.', ms=14, label='Observaciones')
leg = plt.legend(ncol=4, frameon=True, shadow=False, loc=9, edgecolor='k')
frame = leg.get_frame()
frame.set_facecolor('0.9')
plt.ylabel(r'$y=f(x) + \eta$')
plt.ylabel(r'$y=f(x)$')
plt.xlim(t[0], t[-1])
plt.title('GP usando {}% de las observaciones - No entrenado'.format(int(prop*100)))
plt.tight_layout()

## Entrenamiento

In [ ]:
# fixed args of function
args = (y_obs, t_obs)

# initial point
params0 = np.asarray([1, .1, 1])
X0 = np.log(params0)

print('Condicion inicial optimizador: ', params0)

time_GP = time.time()
X_opt, f_GP, data = fmin(
    like_SE,
    X0,
    None,
    args,
    approx_grad=True,
    disp=1,
    factr=0.00000001 / (2.22E-12),
    maxiter=1000)
time_GP = time.time() - time_GP

print("Tiempo entrenamiento {:.4f} (s)".format(time_GP))

sigma_n_GP_opt, gamma_opt, sigma_opt = np.exp(X_opt)
print('Hiperparametros encontrados: ', np.exp(X_opt), 'NLL: ', f_GP)

In [ ]:
# process prior covariance 
cov = K_SE(t, t, gamma=gamma_opt, sigma=sigma_opt)

# observations prior covariance 
cov_obs = K_SE(t_obs, t_obs, gamma=gamma_opt, sigma=sigma_opt)

# K(x*, x) kernel evaluation between obs and test points
K_star = K_SE(t, t_obs, gamma=gamma_opt, sigma=sigma_opt)

# cholesky
cGg = np.linalg.cholesky(cov_obs + sigma_n_GP_opt**2 * np.identity(n_obs))

# inverse covariance matrix (gram)
invGramg = np.linalg.inv(cGg.T) @ np.linalg.inv(cGg)

# temporal matrix K(x*, x) (K(x, x) + sigma * I)^-1
temp = K_star @ invGramg
# posterior mean
m_post = temp @ y_obs
# posterior covariance
sigma_post = np.diag(cov - temp @ K_star.T)
std_dev = np.sqrt(sigma_post)

# plot estimation, obs and real data
plt.plot(t, m_post, c='#00BFFF', lw=4, label='Media de la posterior')
plt.fill_between(
    t,
    m_post - 2 * std_dev,
    m_post + 2 * std_dev,
    facecolor='blue',
    alpha=0.2,
    label='95% IC')
plt.plot(t, y, color='black', ls='--', lw=2, label='Señal real')
plt.plot(t_obs, y_obs, 'r.', ms=14, label='Observaciones')
leg = plt.legend(ncol=4, frameon=True, shadow=False, loc=9, edgecolor='k')
frame = leg.get_frame()
frame.set_facecolor('0.9')
plt.ylabel(r'$y=f(x) + \eta$')
plt.ylabel(r'$y=f(x)$')
plt.xlim(t[0], t[-1])
plt.title('GP usando {}% de las observaciones - Entrenado'.format(int(prop*100)))
plt.tight_layout()

## Señal más grande

In [ ]:
np.random.seed(666)

n_points = 10000
t = np.linspace(0, 250, n_points)
b = np.array([
    9.877821e1, 1.049727e-2, 1.004899e2, 6.748111e1, 2.312977e1, 7.19945e1,
    1.789980e2, 1.838938e1
])
y = b[0] * np.exp(-b[1] * t) + b[2] * np.exp(
    -(t - b[3])**2 / b[4]**2) + b[5] * np.exp(-(t - b[6])**2 / b[7]**2)
y = (y - y.mean()) / y.std()
y_real = y.copy()

percent = 0.1
i_obs = np.random.choice(
    np.arange(0, n_points, 1), int(percent * n_points), replace=False)

###########################
y += np.random.normal(scale=0.2, size=n_points)
###########################

y_obs = y[i_obs]
t_obs = t[i_obs]
n_obs = len(y_obs)

plt.plot(t, y_real)
plt.scatter(t_obs, y_obs, c='r')

In [ ]:
# fixed args of function
args = (y_obs, t_obs)

# initial point
params0 = np.asarray([1, .1, 1])
X0 = np.log(params0)

print('Condicion inicial optimizador: ', params0)

time_GP = time.time()
X_opt, f_GP, data = fmin(
    like_SE,
    X0,
    None,
    args,
    approx_grad=True,
    disp=1,
    factr=0.00000001 / (2.22E-12),
    maxiter=1000)
time_GP = time.time() - time_GP

print("Tiempo entrenamiento {:.4f} (s)".format(time_GP))

sigma_n_GP_opt, gamma_opt, sigma_opt = np.exp(X_opt)
print('Hiperparametros encontrados: ', np.exp(X_opt), 'NLL: ', f_GP)

In [ ]:
# process prior covariance 
cov = K_SE(t, t, gamma=gamma_opt, sigma=sigma_opt)

# observations prior covariance 
cov_obs = K_SE(t_obs, t_obs, gamma=gamma_opt, sigma=sigma_opt)

# K(x*, x) kernel evaluation between obs and test points
K_star = K_SE(t, t_obs, gamma=gamma_opt, sigma=sigma_opt)

# cholesky
cGg = np.linalg.cholesky(cov_obs + sigma_n_GP_opt**2 * np.identity(n_obs))

# inverse covariance matrix (gram)
invGramg = np.linalg.inv(cGg.T) @ np.linalg.inv(cGg)

# temporal matrix K(x*, x) (K(x, x) + sigma * I)^-1
temp = K_star @ invGramg
# posterior mean
m_post = temp @ y_obs
# posterior covariance
sigma_post = np.diag(cov - temp @ K_star.T)
std_dev = np.sqrt(sigma_post)

# plot estimation, obs and real data
plt.plot(t, m_post, c='#00BFFF', lw=4, label='Media de la posterior')
plt.fill_between(
    t,
    m_post - 2 * std_dev,
    m_post + 2 * std_dev,
    facecolor='blue',
    alpha=0.2,
    label='95% IC')
plt.plot(t, y_real, color='black', ls='--', lw=2, label='Señal real')
plt.plot(t_obs, y_obs, 'r.', ms=5, label='Observaciones', alpha=0.5)
leg = plt.legend(ncol=4, frameon=True, shadow=False, loc=9, edgecolor='k')
frame = leg.get_frame()
frame.set_facecolor('0.9')
plt.ylabel(r'$y=f(x) + \eta$')
plt.ylabel(r'$y=f(x)$')
plt.xlim(t[0], t[-1])
plt.title('GP usando {}% de las observaciones - Entrenado'.format(int(percent*100)))
plt.tight_layout()